# SAAF Notebook

This Jupyter Notebook provides an interactive platform for FaaS development. 

In [1]:
# Imports and setup for the notebook.
import os
import sys
sys.path.append(os.path.realpath('..'))
from platforms.jupyter.interactive_helpers import *

# Configure your function here details here.
config = {
    "lambdaRoleARN": "arn:aws:iam::616835888336:role/service-role/simple_microservice_role"
}

# Write Functions

Any function with the @cloud_function decorator will be uploaded to the cloud. Define platforms and memory settings in the decorator. Functions are tested locally and must run sucessfully before being deployed.

In [2]:
#
# A Hello World Function
#
@cloud_function(platforms=[Platform.AWS], memory=512, config=config)
def helloWorld(request, context):
    import json
    import logging
    from Inspector import Inspector
    import time
    
    # Import the module and collect data 
    inspector = Inspector()
    inspector.inspectAll()

    # Add custom message and finish the function
    if ('name' in request):
        inspector.addAttribute("message", "JOW! " + str(request['name']) + "!")
    else:
        inspector.addAttribute("message", "World!")
    
    inspector.inspectAllDeltas()
    return inspector.finish()

In [ ]:
#
# A Pi Calculator
#
@cloud_function(platforms=[Platform.AWS], memory=512, config=config, references=[helloWorld])
def calcPi(request, context):
    import json
    import logging
    from Inspector import Inspector
    import time
    
    # Import the module and collect data
    inspector = Inspector()
    inspector.inspectAll()
    
    # Calculate digits and return
    digits = int(request["digits"])
    
    # Pi Digits function from
    # https://www.geeksforgeeks.org/calculate-pi-with-python/ 
    k = 1
    s = 0
    for i in range(digits):
        if i % 2 == 0:
            s += 4 / k
        else:
            s -= 4 / k
        k += 2
    
    inspector.addAttribute("digits", s)
    
    request = {
        "name": "PI Function"
    }
    
    helloString = helloWorld(request, None)
    inspector.addAttribute("hello", helloString['message'])
    
    inspector.inspectAllDeltas()
    return inspector.finish()
    
test(function=calcPi, payload={"digits": 1000}, config=config)

# Execute Experiments

Use FaaS Runner to execute complex FaaS Experiments.

In [ ]:
experiment = {
	"callWithCLI": True,
  "callAsync": False,
  "memorySettings": [],
  "payloads": [{"digits": 1000000}],
  "runs": 200,
  "threads": 50,
  "iterations": 2,
  "sleepTime": 5,
  "randomSeed": 42,
  "outputGroups": [],
  "outputRawOfGroup": [],
  "showAsList": [],
  "showAsSum": ["newcontainer"],
  "ignoreFromAll": ["zAll", "version", "linuxVersion", "hostname"],
  "ignoreFromGroups": ["1_run_id", "2_thread_id", "cpuModel", "cpuIdle", "cpuIowait", "cpuIrq", "cpuKrn", "cpuNice", "cpuSoftIrq", "cpuUsr"],
  "ignoreByGroup": {
  "containerID": ["containerID"],
  "cpuType": ["cpuType"],
  "vmID": ["vmID"]
  },
  "invalidators": {},
  "removeDuplicateContainers": False,
  "overlapFilter": "functionName",
  "openCSV": False,
  "combineSheets": True,
  "warmupBuffer": 0
}

results = run_experiment(function=calcPi, platform=Platform.AWS, experiment=experiment, config=config)
results

# Process Results

FaaS Runner experiment results are parsed into a Pandas dataframe. This the flexibility to perform any kind of data processing that you would like.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# Average Runtime
#np.average(results['userRuntime'])

# Histogram of Runtime
plt.hist(results['userRuntime'], 10)

# Scatter plot of CPU Profile
#plt.plot(results['cpuUsrDelta'], results['cpuIdleDelta'], 'o', color='black')

In [ ]:
@cloud_function(platforms=[Platform.AWS], memory=512, config=config, references=[calcPi])
def calcPiDistributor(request, context):
    import json
    import logging
    from Inspector import Inspector
    import time
    import threading
    
    # Import the module and collect data
    inspector = Inspector()
    inspector.inspectAll()

    threads = []
    for x in range(request['runs']):
        threads.append(threading.Thread(target=calcPi, args=({"digits": 10000}, None,)))
    for thread in threads:
        thread.start()
    for thread in threads:
        thread.join()
    
    inspector.inspectAllDeltas()
    return inspector.finish()

test(function=calcPiDistributor, payload={"runs": 20}, config=config)